# Distracted Driver Detection using deep learning

#### Aniruddha Humane  4321
#### Neha Prabhune         4341
#### Vasanti Sathe            4347 

#### Guide - Prof. D. T. Mane

Importing Keras

In [34]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
K.set_image_dim_ordering('th')

Creating a very small neural network from scratch due to limited time
###### Details of neural network

#### Input layer
- no of neurons = no of pixels = 3x150x150 = 64500 neurons

#### Convolutional layer
- 32 filters (3x3)
- input (3*150*150)
- Activation: ReLU
- input no of neurons = 64500 neurons
- output no of neurons = no of filters = 32

#### Max Pooling layer
- downscaling by (2x2)
- input no of neurons = 32
- output no of neurons = 16

#### Convolutional layer
- 32 filters (3x3)
- Activation: ReLU
- input no of neurons = 16
- output no of neurons = 32

#### Max Pooling layer
- downscaling by (2x2)
- input no of neurons = 32
- output no of neurons = 16

#### Convolutional layer
- 64 filters (3x3)
- Activation: ReLU
- input no of neurons = 16
- output no of neurons = 64

#### Max Pooling layer
- downscaling by (2x2)
- input no of neurons = 64
- output no of neurons = 32

#### Fully connected layer
- 64 neurons
- Activation: ReLU
- input no of neurons = 32
- output no of neurons = 64
- dropout = 50%

#### Fully connected output layer
- 10 neurons
- Activation: SoftMax
- Input no of neurons = 32
- output no of neurons = 10

In [15]:
model=Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3,150,150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# the model so far outputs 3D feature maps (height, width, features)
model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64)) # 64 neurons
model.add(Activation('relu'))
model.add(Dropout(0.5)) # drop 50% of neurons

# output layer: classify to 10 driver's states
model.add(Dense(10))
model.add(Activation('softmax'))

Configuring the learning process (Compilation)

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

Setting up _*Augmentation Configuration*_ for training images

In [17]:
train_datagen=ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

scaling validation images _(no augmentation is applied on validation images as we need to test our model)_

In [18]:
test_datagen=ImageDataGenerator(rescale=1.0/255)

Generating training data
- read images from directory data/train
- Until we have our training data generating indefininte number of _augmented batches of size = 32_

In [25]:
train_generator=train_datagen.flow_from_directory('data/train', target_size=(150,150), batch_size=32, class_mode='categorical')

Found 18574 images belonging to 10 classes.


Similar generator for validation set

In [24]:
validation_generator=test_datagen.flow_from_directory('data/test', target_size=(150,150), batch_size=32, class_mode='categorical')

Found 3850 images belonging to 10 classes.


Training the CNN on the training and test set generated

In [35]:
model.fit_generator(train_generator, samples_per_epoch=20924, nb_epoch=20, validation_data=validation_generator, nb_val_samples=800)

Epoch 1/20


E:\Software\Conda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
E:\Software\Conda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=653, epochs=20, validation_steps=800)`
  """Entry point for launching an IPython kernel.


StopIteration: Could not import PIL.Image. The use of `array_to_img` requires PIL.

Save generated weights from training to obtain results later

In [ ]:
model.save_weights('driver_state_detection_small_CNN.h5')